## Preprocessing
Requirements:
- Modular for: 
  - Transform type
  - train/val/test directories
- General template: 
  - Isolate train/val/test datapoints
  - Transform each datapoint
  - Save to respective directories

Notes:
- ENSTDkAm and ENSTDkCl = non-synthesized recording directories

## Imports

In [1]:
import os
import pretty_midi as pm
import librosa as lr
import madmom as mm
import numpy as np

## Constants

In [2]:
DATASET_DIR = '../datasets/maps/'
MIN_MIDI = 21
MAX_MIDI = 108
WINDOW_SIZE = 5
SPLITS_DIR = '../splits/'

## Helpers

In [3]:
def transform_wav(track_path):
    # Log Spect
    log_spect = mm.audio.spectrogram.LogarithmicFilteredSpectrogram(
        track_path,
        num_bands=48,
        num_channels=1,
        sr=44100,
        unique_filters=True,
        norm_filters=True,
        frame_size=4096,
        fft_size=4096,
        fmin=30,
        fmax=8000.0,
        fref=440.0,
        circular_shift=False,
        fps=31.25)
    
    # Normalize, Rescale
    log_spect = np.nan_to_num(log_spect)
    log_spect = np.abs(log_spect)
    log_spect /= np.max(log_spect, axis=0)
    
    # Generate Windows:
    log_spect = np.pad(
        log_spect,
        ((WINDOW_SIZE//2, WINDOW_SIZE//2), (0,0)), 
        'constant')
    
    windows = []
    for i in range(log_spect.shape[0] - WINDOW_SIZE + 1):
        w = log_spect[i:i+WINDOW_SIZE,:]
        windows.append(w)

    return np.array(windows)
    
def generate_expected(midi, times):
    expected = midi.get_piano_roll(fs=44100, times=times)[MIN_MIDI:MAX_MIDI+1].T
    expected[expected > 0] = 1
    return expected

## Transform Datapoints
(735 .wav files in local sample)

In [4]:
total_processed = 0
model_split_dir = SPLITS_DIR + 'config-2_logfilt_baseline/'
train_dir = model_split_dir + 'train/'
test_dir = model_split_dir + 'test/'
expect_dir = model_split_dir + 'expect/'
partition = {}
partition['train'] = []
partition['test'] = []

if not os.path.exists(model_split_dir):
    os.mkdir(model_split_dir)
if not os.path.exists(train_dir):
    os.mkdir(train_dir)
if not os.path.exists(test_dir):
    os.mkdir(test_dir)
if not os.path.exists(expect_dir):
    os.mkdir(expect_dir)

In [5]:
for subdir_name in os.listdir(DATASET_DIR):
    subdir_path = os.path.join(DATASET_DIR, subdir_name)
        
    # Check if dir, not file.
    if not os.path.isdir(subdir_path):
        continue
        
    for dir_parent, dir_name, file_names in os.walk(subdir_path):
        for name in file_names:
            if name.endswith('.wav'):
                track_name = name.split('.wav')[0]
                midi_name = track_name + '.mid'
                txt_name = track_name + '.txt'
                
                if midi_name in file_names:
                
                    # To transform
                    track_path = os.path.join(dir_parent, name)
                    
            # DELETE
                    print "Processing " + track_path

                    # To generate ground truth
                    midi_path = os.path.join(dir_parent, midi_name)
                    pm_midi = pm.PrettyMIDI(midi_path)

                    # Transform and Gen Ground Truth
                    np_input = transform_wav(track_path)
                    times = lr.frames_to_time(
                                np.arange(np_input.shape[0]),
                                sr=44100,
                                hop_length=441.0)
                    np_output = generate_expected(
                                pm_midi,
                                times)
                    
                    datapoint_id = track_name + '.dat'
                    set_type = train_dir 
                    if subdir_name != "NO":
                        partition['train'].append(datapoint_id)
                    else:
                        set_type = test_dir
                        partition['test'].append(datapoint_id)
                    
                    mm_input = np.memmap(
                        filename=set_type + datapoint_id,
                        mode='w+',
                        shape=np_input.shape)
                    mm_input[:] = np_input[:]
                    del mm_input
                    
                    mm_output = np.memmap(
                        filename=expect_dir + datapoint_id,
                        mode='w+',
                        shape=np_output.shape)
                    mm_output = np_output[:]
                    del mm_output
                    
            # DELETE
                    print np_input.shape
                    print np_output.shape
        

Processing ../datasets/maps/NO/MAPS_ISOL_NO_M_S1_M82_SptkBGAm.wav
(154, 5, 229)
(154, 88)
Processing ../datasets/maps/NO/MAPS_ISOL_NO_P_S0_M104_SptkBGAm.wav
(141, 5, 229)
(141, 88)
Processing ../datasets/maps/NO/MAPS_ISOL_NO_P_S1_M22_SptkBGAm.wav
(154, 5, 229)
(154, 88)
Processing ../datasets/maps/NO/MAPS_ISOL_NO_P_S1_M77_SptkBGAm.wav
(154, 5, 229)
(154, 88)
Processing ../datasets/maps/NO/MAPS_ISOL_NO_M_S1_M85_SptkBGAm.wav
(154, 5, 229)
(154, 88)
Processing ../datasets/maps/NO/MAPS_ISOL_NO_F_S0_M68_SptkBGAm.wav
(141, 5, 229)
(141, 88)
Processing ../datasets/maps/NO/MAPS_ISOL_NO_P_S0_M58_SptkBGAm.wav
(141, 5, 229)
(141, 88)
Processing ../datasets/maps/NO/MAPS_ISOL_NO_M_S0_M52_SptkBGAm.wav
(141, 5, 229)
(141, 88)
Processing ../datasets/maps/NO/MAPS_ISOL_NO_M_S0_M107_SptkBGAm.wav
(141, 5, 229)
(141, 88)
Processing ../datasets/maps/NO/MAPS_ISOL_NO_P_S1_M102_SptkBGAm.wav
(154, 5, 229)
(154, 88)
Processing ../datasets/maps/NO/MAPS_ISOL_NO_M_S1_M90_SptkBGAm.wav
(154, 5, 229)
(154, 88)
Process

(141, 5, 229)
(141, 88)
Processing ../datasets/maps/NO/MAPS_ISOL_NO_M_S1_M63_SptkBGAm.wav
(154, 5, 229)
(154, 88)
Processing ../datasets/maps/NO/MAPS_ISOL_NO_F_S1_M91_SptkBGAm.wav
(154, 5, 229)
(154, 88)
Processing ../datasets/maps/NO/MAPS_ISOL_NO_F_S0_M101_SptkBGAm.wav
(141, 5, 229)
(141, 88)
Processing ../datasets/maps/NO/MAPS_ISOL_NO_M_S0_M29_SptkBGAm.wav
(141, 5, 229)
(141, 88)
Processing ../datasets/maps/NO/MAPS_ISOL_NO_F_S1_M69_SptkBGAm.wav
(154, 5, 229)
(154, 88)
Processing ../datasets/maps/NO/MAPS_ISOL_NO_F_S1_M96_SptkBGAm.wav
(154, 5, 229)
(154, 88)
Processing ../datasets/maps/NO/MAPS_ISOL_NO_P_S0_M71_SptkBGAm.wav
(141, 5, 229)
(141, 88)
Processing ../datasets/maps/NO/MAPS_ISOL_NO_F_S0_M106_SptkBGAm.wav
(141, 5, 229)
(141, 88)
Processing ../datasets/maps/NO/MAPS_ISOL_NO_F_S0_M53_SptkBGAm.wav
(141, 5, 229)
(141, 88)
Processing ../datasets/maps/NO/MAPS_ISOL_NO_M_S1_M46_SptkBGAm.wav
(154, 5, 229)
(154, 88)
Processing ../datasets/maps/NO/MAPS_ISOL_NO_P_S0_M64_SptkBGAm.wav
(141, 5,

(154, 5, 229)
(154, 88)
Processing ../datasets/maps/NO/MAPS_ISOL_NO_F_S0_M57_SptkBGAm.wav
(141, 5, 229)
(141, 88)
Processing ../datasets/maps/NO/MAPS_ISOL_NO_M_S0_M80_SptkBGAm.wav
(141, 5, 229)
(141, 88)
Processing ../datasets/maps/NO/MAPS_ISOL_NO_M_S0_M78_SptkBGAm.wav
(141, 5, 229)
(141, 88)
Processing ../datasets/maps/NO/MAPS_ISOL_NO_M_S0_M87_SptkBGAm.wav
(141, 5, 229)
(141, 88)
Processing ../datasets/maps/NO/MAPS_ISOL_NO_F_S0_M61_SptkBGAm.wav
(141, 5, 229)
(141, 88)
Processing ../datasets/maps/NO/MAPS_ISOL_NO_F_S0_M34_SptkBGAm.wav
(141, 5, 229)
(141, 88)
Processing ../datasets/maps/NO/MAPS_ISOL_NO_P_S1_M79_SptkBGAm.wav
(154, 5, 229)
(154, 88)
Processing ../datasets/maps/NO/MAPS_ISOL_NO_M_S1_M26_SptkBGAm.wav
(154, 5, 229)
(154, 88)
Processing ../datasets/maps/NO/MAPS_ISOL_NO_F_S0_M66_SptkBGAm.wav
(141, 5, 229)
(141, 88)
Processing ../datasets/maps/NO/MAPS_ISOL_NO_F_S0_M33_SptkBGAm.wav
(141, 5, 229)
(141, 88)
Processing ../datasets/maps/NO/MAPS_ISOL_NO_M_S1_M74_SptkBGAm.wav
(154, 5, 2

(207, 5, 229)
(207, 88)
Processing ../datasets/maps/TR1/MAPS_ISOL_TR1_P_S1_M94_SptkBGAm.wav
(220, 5, 229)
(220, 88)
Processing ../datasets/maps/TR1/MAPS_ISOL_TR1_F_S0_M99_SptkBGAm.wav
(207, 5, 229)
(207, 88)
Processing ../datasets/maps/TR1/MAPS_ISOL_TR1_F_S0_M33_SptkBGAm.wav
(207, 5, 229)
(207, 88)
Processing ../datasets/maps/TR1/MAPS_ISOL_TR1_M_S1_M74_SptkBGAm.wav
(220, 5, 229)
(220, 88)
Processing ../datasets/maps/TR1/MAPS_ISOL_TR1_P_S1_M104_SptkBGAm.wav
(220, 5, 229)
(220, 88)
Processing ../datasets/maps/TR1/MAPS_ISOL_TR1_P_S0_M49_SptkBGAm.wav
(207, 5, 229)
(207, 88)
Processing ../datasets/maps/TR1/MAPS_ISOL_TR1_F_S1_M51_SptkBGAm.wav
(220, 5, 229)
(220, 88)
Processing ../datasets/maps/TR1/MAPS_ISOL_TR1_P_S1_M21_SptkBGAm.wav
(220, 5, 229)
(220, 88)
Processing ../datasets/maps/TR1/MAPS_ISOL_TR1_P_S1_M99_SptkBGAm.wav
(220, 5, 229)
(220, 88)
Processing ../datasets/maps/TR1/MAPS_ISOL_TR1_F_S0_M48_SptkBGAm.wav
(207, 5, 229)
(207, 88)
Processing ../datasets/maps/TR1/MAPS_ISOL_TR1_F_S1_M60_

(92, 5, 229)
(92, 88)
Processing ../datasets/maps/ST/MAPS_ISOL_ST_F_S0_M98_SptkBGAm.wav
(79, 5, 229)
(79, 88)
Processing ../datasets/maps/ST/MAPS_ISOL_ST_F_S0_M102_SptkBGAm.wav
(79, 5, 229)
(79, 88)
Processing ../datasets/maps/ST/MAPS_ISOL_ST_F_S0_M67_SptkBGAm.wav
(79, 5, 229)
(79, 88)
Processing ../datasets/maps/ST/MAPS_ISOL_ST_P_S1_M87_SptkBGAm.wav
(92, 5, 229)
(92, 88)
Processing ../datasets/maps/ST/MAPS_ISOL_ST_F_S0_M35_SptkBGAm.wav
(79, 5, 229)
(79, 88)
Processing ../datasets/maps/ST/MAPS_ISOL_ST_F_S0_M105_SptkBGAm.wav
(79, 5, 229)
(79, 88)
Processing ../datasets/maps/ST/MAPS_ISOL_ST_M_S1_M72_SptkBGAm.wav
(92, 5, 229)
(92, 88)
Processing ../datasets/maps/ST/MAPS_ISOL_ST_M_S0_M48_SptkBGAm.wav
(79, 5, 229)
(79, 88)
Processing ../datasets/maps/ST/MAPS_ISOL_ST_P_S0_M45_SptkBGAm.wav
(79, 5, 229)
(79, 88)
Processing ../datasets/maps/ST/MAPS_ISOL_ST_M_S1_M67_SptkBGAm.wav
(92, 5, 229)
(92, 88)
Processing ../datasets/maps/ST/MAPS_ISOL_ST_M_S1_M98_SptkBGAm.wav
(92, 5, 229)
(92, 88)
Processi

(92, 5, 229)
(92, 88)
Processing ../datasets/maps/ST/MAPS_ISOL_ST_F_S1_M108_SptkBGAm.wav
(92, 5, 229)
(92, 88)
Processing ../datasets/maps/ST/MAPS_ISOL_ST_F_S0_M86_SptkBGAm.wav
(79, 5, 229)
(79, 88)
Processing ../datasets/maps/ST/MAPS_ISOL_ST_F_S0_M79_SptkBGAm.wav
(79, 5, 229)
(79, 88)
Processing ../datasets/maps/ST/MAPS_ISOL_ST_M_S0_M51_SptkBGAm.wav
(79, 5, 229)
(79, 88)
Processing ../datasets/maps/ST/MAPS_ISOL_ST_F_S1_M43_SptkBGAm.wav
(92, 5, 229)
(92, 88)
Processing ../datasets/maps/ST/MAPS_ISOL_ST_F_S1_M88_SptkBGAm.wav
(92, 5, 229)
(92, 88)
Processing ../datasets/maps/ST/MAPS_ISOL_ST_F_S1_M77_SptkBGAm.wav
(92, 5, 229)
(92, 88)
Processing ../datasets/maps/ST/MAPS_ISOL_ST_M_S0_M30_SptkBGAm.wav
(79, 5, 229)
(79, 88)
Processing ../datasets/maps/ST/MAPS_ISOL_ST_F_S1_M25_SptkBGAm.wav
(92, 5, 229)
(92, 88)
Processing ../datasets/maps/ST/MAPS_ISOL_ST_M_S0_M37_SptkBGAm.wav
(79, 5, 229)
(79, 88)
Processing ../datasets/maps/ST/MAPS_ISOL_ST_M_S0_M62_SptkBGAm.wav
(79, 5, 229)
(79, 88)
Processin

(92, 5, 229)
(92, 88)
Processing ../datasets/maps/ST/MAPS_ISOL_ST_F_S1_M29_SptkBGAm.wav
(92, 5, 229)
(92, 88)
Processing ../datasets/maps/ST/MAPS_ISOL_ST_F_S1_M84_SptkBGAm.wav
(92, 5, 229)
(92, 88)
Processing ../datasets/maps/ST/MAPS_ISOL_ST_P_S0_M63_SptkBGAm.wav
(79, 5, 229)
(79, 88)
Processing ../datasets/maps/ST/MAPS_ISOL_ST_M_S1_M53_SptkBGAm.wav
(92, 5, 229)
(92, 88)
Processing ../datasets/maps/ST/MAPS_ISOL_ST_F_S0_M46_SptkBGAm.wav
(79, 5, 229)
(79, 88)
Processing ../datasets/maps/ST/MAPS_ISOL_ST_M_S1_M54_SptkBGAm.wav
(92, 5, 229)
(92, 88)
Processing ../datasets/maps/ST/MAPS_ISOL_ST_P_S0_M76_SptkBGAm.wav
(79, 5, 229)
(79, 88)
Processing ../datasets/maps/ST/MAPS_ISOL_ST_P_S0_M23_SptkBGAm.wav
(79, 5, 229)
(79, 88)
Processing ../datasets/maps/ST/MAPS_ISOL_ST_F_S1_M91_SptkBGAm.wav
(92, 5, 229)
(92, 88)
Processing ../datasets/maps/ST/MAPS_ISOL_ST_P_S0_M71_SptkBGAm.wav
(79, 5, 229)
(79, 88)
Processing ../datasets/maps/ST/MAPS_ISOL_ST_F_S1_M69_SptkBGAm.wav
(92, 5, 229)
(92, 88)
Processing

(220, 5, 229)
(220, 88)
Processing ../datasets/maps/TR2/MAPS_ISOL_TR2_F_S0_M51_SptkBGAm.wav
(207, 5, 229)
(207, 88)
Processing ../datasets/maps/TR2/MAPS_ISOL_TR2_P_S1_M49_SptkBGAm.wav
(220, 5, 229)
(220, 88)
Processing ../datasets/maps/TR2/MAPS_ISOL_TR2_P_S0_M101_SptkBGAm.wav
(207, 5, 229)
(207, 88)
Processing ../datasets/maps/TR2/MAPS_ISOL_TR2_M_S0_M81_SptkBGAm.wav
(207, 5, 229)
(207, 88)
Processing ../datasets/maps/TR2/MAPS_ISOL_TR2_P_S0_M66_SptkBGAm.wav
(207, 5, 229)
(207, 88)
Processing ../datasets/maps/TR2/MAPS_ISOL_TR2_P_S0_M34_SptkBGAm.wav
(207, 5, 229)
(207, 88)
Processing ../datasets/maps/TR2/MAPS_ISOL_TR2_F_S1_M73_SptkBGAm.wav
(220, 5, 229)
(220, 88)
Processing ../datasets/maps/TR2/MAPS_ISOL_TR2_F_S1_M34_SptkBGAm.wav
(220, 5, 229)
(220, 88)
Processing ../datasets/maps/TR2/MAPS_ISOL_TR2_P_S0_M79_SptkBGAm.wav
(207, 5, 229)
(207, 88)
Processing ../datasets/maps/TR2/MAPS_ISOL_TR2_F_S1_M66_SptkBGAm.wav
(220, 5, 229)
(220, 88)
Processing ../datasets/maps/TR2/MAPS_ISOL_TR2_M_S1_M80_

In [6]:
total_processed

0

In [7]:
partition['train']

['MAPS_ISOL_TR1_F_S1_M81_SptkBGAm.dat',
 'MAPS_ISOL_TR1_P_S0_M66_SptkBGAm.dat',
 'MAPS_ISOL_TR1_F_S1_M79_SptkBGAm.dat',
 'MAPS_ISOL_TR1_F_S0_M106_SptkBGAm.dat',
 'MAPS_ISOL_TR1_F_S1_M39_SptkBGAm.dat',
 'MAPS_ISOL_TR1_M_S1_M44_SptkBGAm.dat',
 'MAPS_ISOL_TR1_M_S1_M72_SptkBGAm.dat',
 'MAPS_ISOL_TR1_P_S1_M38_SptkBGAm.dat',
 'MAPS_ISOL_TR1_M_S0_M78_SptkBGAm.dat',
 'MAPS_ISOL_TR1_M_S0_M38_SptkBGAm.dat',
 'MAPS_ISOL_TR1_F_S0_M107_SptkBGAm.dat',
 'MAPS_ISOL_TR1_M_S1_M33_SptkBGAm.dat',
 'MAPS_ISOL_TR1_F_S0_M21_SptkBGAm.dat',
 'MAPS_ISOL_TR1_P_S1_M94_SptkBGAm.dat',
 'MAPS_ISOL_TR1_F_S0_M99_SptkBGAm.dat',
 'MAPS_ISOL_TR1_F_S0_M33_SptkBGAm.dat',
 'MAPS_ISOL_TR1_M_S1_M74_SptkBGAm.dat',
 'MAPS_ISOL_TR1_P_S1_M104_SptkBGAm.dat',
 'MAPS_ISOL_TR1_P_S0_M49_SptkBGAm.dat',
 'MAPS_ISOL_TR1_F_S1_M51_SptkBGAm.dat',
 'MAPS_ISOL_TR1_P_S1_M21_SptkBGAm.dat',
 'MAPS_ISOL_TR1_P_S1_M99_SptkBGAm.dat',
 'MAPS_ISOL_TR1_F_S0_M48_SptkBGAm.dat',
 'MAPS_ISOL_TR1_F_S1_M60_SptkBGAm.dat',
 'MAPS_ISOL_TR1_P_S1_M42_SptkBGAm.dat

## Load np arrays